# Quantiles
How do ENSO quantiles change over time?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import scipy.stats
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import cartopy.util
import copy

# Import custom modules
import src.utils
from src.XRO import XRO, xcorr

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Load data

In [ ]:
## Load T,h (total)
Th_total = xr.open_dataset(DATA_FP / "cesm" / "Th.nc")

## get windowed data
Th_total = src.utils.get_windowed(Th_total, stride=60)

## seasonal mean
Th_total = Th_total.resample({"time": "QS-DEC"}).mean()

## compute quantiles
quant = Th_total.groupby("time.season").quantile(
    q=[0.05, 0.5, 0.95], dim=["time", "member"]
)
quant = quant.rename({"quantile": "q"})

## compute difference
delta_q = quant - quant.isel(year=0)

## get spread
spread = quant.sel(q=0.95) - quant.sel(q=0.05)
delta_spread = spread - spread.isel(year=0)

## Analysis

### Plot quantiles over time

In [ ]:
VARNAME = "T_3"
SEASON = "DJF"

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.05), c="b")
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.50), c="gray")
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.95), c="r")

ax.plot(
    quant.year,
    delta_spread[VARNAME].sel(season=SEASON),
    ls="--",
    c="k",
)
# ax.plot(delta_q.year, delta_q["T_34"].sel(season=SEASON, q=0.95))

## format
ax.set_xlim([None, 2000])
ax.set_ylim([-0.2, 1])
ax.axhline(0, c="k", lw=0.5)
plt.show()

### Plot PDFs over time

In [ ]:
## specify variable
VARNAME = "T_3"

## specify season
QS = 12

## edges for PDF
edges = np.arange(21, 36, 0.5)

## empty array to hold pdfs
pdfs = []

for year in Th_total.year:

    ## get data
    y = src.utils.sel_month(Th_total.sel(year=year), QS)[VARNAME]

    ## compute pdf
    pdfs.append(src.utils.get_empirical_pdf(y, edges=edges)[0])

## put in dataset
pdfs = xr.Dataset(
    data_vars=dict(
        pdf=(("year", "bin_idx"), np.stack(pdfs, axis=0)),
        lb=("bin_idx", edges[:-1]),
        ub=("bin_idx", edges[1:]),
    ),
    coords=dict(year=Th_total.year.values, bin_idx=np.arange(len(edges) - 1)),
)

colors = sns.color_palette("mako")[::3]

fig, ax = plt.subplots(figsize=(5, 4))

# for year, c in zip([1870, 1970], colors):
ax.stairs(pdfs["pdf"].sel(year=1870), edges=edges, fill=True, alpha=0.3)
ax.stairs(pdfs["pdf"].sel(year=1970), edges=edges, lw=2)

ax.set_xlim([None, 31.5])
plt.show()